# **Computational Theory**
### *Adam Gallagher G00413950*

## **Problem 1: _Binary Words and Operations_**

## **Problem 2: _Fractional Parts of Cube Roots_**

## **Problem 3: _Padding_**

## **Problem 4: _Hashes_**

## **Problem 5: _Passwords_**